In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.10.1


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
lst = []

with open('../data/full_tab_string.txt') as small_pf:

    tmp_list = []
    for line in small_pf:
        line = line.rstrip("\n")
        if line == "":
            lst.append(tmp_list)
            tmp_list = []
        else:
            tmp_list.extend(line.split())

    if tmp_list:  # add last one
        lst.append(tmp_list)
lst = lst[0]

In [5]:
text = ""
with open('../data/full_tab_string.txt') as small_pf:
    text = small_pf.read()

In [6]:
#Can't use set because tab encodings are not hashable types
def unique(list1):
    unique_set = set()
    for x in list1:
        if x not in unique_set:
            unique_set.add(x)
    return unique_set

In [7]:
#list of all tab locations
tab_vocab = unique(lst)

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    num_oov_indices = 1, 
    vocabulary=list(tab_vocab), 
    mask_token=None)
print(ids_from_chars.get_vocabulary())

['[UNK]', 'e11', 'e1', 'B5', 'D13', 'e12', 'B21', 'G10', 'D12', 'D4', 'B3', 'E11', 'E0', 'B9', 'B2', 'A33', 'D1', 'D7', 'D6', 'D2', 'e77', 'B33', 'e24', 'E34', 'A16', 'e28', 'A5', 'A14', 'G19', 'G81', 'B19', 'B51', 'D14', 'B4', 'G13', 'B24', 'A1', 'A10', 'e5', 'e51', 'D10', 'e21', 'G1', 'e10', 'G21', 'B15', 'B11', 'G0', 'A54', 'G9', 'E33', 'A0', 'E14', 'e4', 'B6', 'B53', 'D16', 'B13', 'B17', 'e13', 'A2', 'G40', 'G22', 'G7', 'A15', 'A17', 'B31', 'A77', 'e16', 'D5', 'D77', 'B23', 'E3', 'A6', 'e22', 'G92', 'e2', 'A19', 'G12', 'G8', 'A52', 'D19', 'E77', 'A25', 'A44', 'B18', 'E12', 'E13', 'A45', 'D75', 'B22', 'D52', 'G42', 'B12', 'E2', 'E15', 'G91', 'e7', 'B71', 'G2', 'E5', 'D17', 'E8', 'e18', 'D22', 'D3', 'A75', 'A7', 'e15', 'G11', 'E17', 'B10', 'e19', 'E19', 'A99', 'D20', 'G61', 'E4', 'e14', 'A12', 'G14', 'E55', 'B14', 'G17', 'A13', 'E22', 'G24', 'A22', 'A8', 'A57', 'E9', 'D11', 'E10', 'B7', 'B20', 'A11', 'G23', 'E1', 'E57', 'G4', 'e3', 'G16', 'D25', 'D8', 'A4', 'B1', 'B88', 'B16', 'G3', 

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [10]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids) + " ", axis=-1)

In [11]:
all_ids = ids_from_chars(tf.strings.split(text))

In [12]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
type(ids_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [13]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

D7
G5
B5
e5
e7
D6
B5
G5
e7
e8


In [14]:
seq_length = 10

In [15]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
print(type(sequences))
for seq in sequences.take(1):
  print(chars_from_ids(seq))
  print(type(seq))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
tf.Tensor([b'D7' b'G5' b'B5' b'e5' b'e7' b'D6' b'B5' b'G5' b'e7' b'e8' b'D5'], shape=(11,), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [16]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'D7 G5 B5 e5 e7 D6 B5 G5 e7 e8 D5 '
b'B5 G5 e8 e2 D4 B3 G2 e2 e0 D3 B1 '
b'G2 B1 e0 B1 G2 B0 G0 A2 B1 G2 A0 '
b'B1 G2 A0 G2 A0 A8 A7 A0 D7 G5 B5 '
b'e7 D6 B5 G5 e7 e8 D5 B5 G5 e8 e2 '


In [17]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [18]:
dataset = sequences.map(split_input_target)
dataset



<MapDataset element_spec=(TensorSpec(shape=(10,), dtype=tf.int64, name=None), TensorSpec(shape=(10,), dtype=tf.int64, name=None))>

In [19]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'D7 G5 B5 e5 e7 D6 B5 G5 e7 e8 '
Target: b'G5 B5 e5 e7 D6 B5 G5 e7 e8 D5 '


In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 10), dtype=tf.int64, name=None), TensorSpec(shape=(64, 10), dtype=tf.int64, name=None))>

In [21]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 2048

In [22]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)

    #adjust the dense units size                         
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [23]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 10, 173) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  44288     
                                                                 
 gru (GRU)                   multiple                  14168064  
                                                                 
 dense (Dense)               multiple                  354477    
                                                                 
Total params: 14,566,829
Trainable params: 14,566,829
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [27]:
sampled_indices

array([ 54,  52, 117,  11,   1,  44, 123, 113,   4,  19], dtype=int64)

In [28]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'A7 D2 E2 E2 A4 E2 A5 E2 A4 E2 '

Next Char Predictions:
 b'B6 E14 E4 E11 e11 G21 G17 E19 D13 D2 '


In [29]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [30]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 10, 173)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.1531043, shape=(), dtype=float32)


In [31]:
model.compile(optimizer='adam', loss=loss)

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './tab_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [38]:
EPOCHS = 100

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
248/248 [==============================] - 4s 15ms/step - loss: 0.7515
Epoch 2/100
248/248 [==============================] - 4s 15ms/step - loss: 0.7350
Epoch 3/100
248/248 [==============================] - 4s 14ms/step - loss: 0.7253
Epoch 4/100
248/248 [==============================] - 4s 14ms/step - loss: 0.7159
Epoch 5/100
248/248 [==============================] - 4s 14ms/step - loss: 0.7061
Epoch 6/100
248/248 [==============================] - 4s 14ms/step - loss: 0.6994
Epoch 7/100
248/248 [==============================] - 4s 14ms/step - loss: 0.6954
Epoch 8/100
248/248 [==============================] - 4s 14ms/step - loss: 0.6892
Epoch 9/100
248/248 [==============================] - 4s 14ms/step - loss: 0.6849
Epoch 10/100
248/248 [==============================] - 4s 15ms/step - loss: 0.6803
Epoch 11/100
248/248 [==============================] - 4s 15ms/step - loss: 0.6767
Epoch 12/100
248/248 [==============================] - 4s 15ms/step - loss: 0.6713
E

KeyboardInterrupt: 

In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.

    #USE THIS TO CHANGE WHICH OUTPUTS ARE ALLOWED
    # skip_ids = self.ids_from_chars(omit)[:, None]

    # sparse_mask = tf.SparseTensor(
    #     # Put a -inf at each bad index.
    #     values=[-float('inf')]*len(skip_ids),
    #     indices=skip_ids,
    #     # Match the shape to the vocabulary
    #     dense_shape=[len(ids_from_chars.get_vocabulary())])
    # self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  #@tf.function
  def generate_one_step(self, inputs, omit, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]

    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.

    skip_ids = self.ids_from_chars(omit)[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [41]:
from MidiToTabOptions import *
tab_midi_dict = generate_tab_midi_pairs()

In [43]:
#Array of Midi notes
notes = [58, 60, 61 , 63, 60, 56, 58]
states = None

#First tab note, hard coded
#base_tab = 'B11 '
base_tab = 'D7 '
#base_tab = 'D12 '

one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
#Dynamically starting input 
append_tab = tf.constant([base_tab])
for note in notes:

    #Finds all tab options for the current note and 
    #removes them from the omitted option list in the OneStep function
    options = ['[UNK]']
    options.extend(tab_midi_dict[note])
    omit = list(set(ids_from_chars.get_vocabulary()) - set(options))
    omit.insert(0, '[UNK]')

    omit_sorted = []
    for item in ids_from_chars.get_vocabulary():
        if item in omit:
            omit_sorted.append(item)

    #generates the next predicted tab note out of the set of possible options
    #one_step_model = OneStep(model, chars_from_ids, ids_from_chars, omit_sorted)

    #bootstrap the predictions 
    bootstrap_list = []    
    for i in range(100):
        next_char, states = one_step_model.generate_one_step(append_tab, omit_sorted, states=states)    
        bootstrap_list.append(next_char.numpy()[0].decode('utf-8'))
    next_tab = max(set(bootstrap_list), key=bootstrap_list.count)


    #append new tab to input list
    append_tab = tf.constant([append_tab.numpy()[0].decode('utf-8') + next_tab + ' '])
    print(append_tab)

tf.Tensor([b'D7 G3 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G3 D10 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G3 D10 B2 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G3 D10 B2 G8 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G3 D10 B2 G8 G5 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G3 D10 B2 G8 G5 G1 '], shape=(1,), dtype=string)
tf.Tensor([b'D7 G3 D10 B2 G8 G5 G1 G3 '], shape=(1,), dtype=string)
